In [ ]:
from get_data import *
from learn import *
from tuning import *
from analysis import *
from binance.client import Client
import numpy as np
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd
from backtesting import Backtest
pd.options.mode.chained_assignment = None  # default='warn'

### Get data

In [ ]:


kline = get_klines(Client.KLINE_INTERVAL_4HOUR,'2019-01-01','2022-12-02')
OHLC = get_OHLC(kline)
tech_index = get_tech(OHLC,period=14)
OHLC_label = get_label(OHLC)
feature_idx = get_features(tech_index)


OHLC_hasFeature = OHLC_label.label_2[OHLC_label.index.isin(feature_idx.index)].shift(-1)
OHLC_hasFeature = OHLC_hasFeature[1:-6]

feature_idx_sel = feature_idx.shift(periods=1)[1:-6]
feature_cnt = feature_idx_sel.apply(pd.Series.value_counts)
print("feature:")
print(feature_cnt)
splitpoint=  OHLC_hasFeature.index.get_loc('2021-10-01 00:00:00')

# indicator
X_train = feature_idx_sel[:splitpoint].to_numpy()
y_train = OHLC_hasFeature[:splitpoint].to_numpy()
X_test = feature_idx_sel[splitpoint:].to_numpy()
y_test = OHLC_hasFeature[splitpoint:].to_numpy()

# return
OHLC_return = OHLC_label['return'][:-6].to_numpy()
OHLC_window = np.zeros((len(OHLC_return)-15,14))
for i in range(15,len(OHLC_return)):
    OHLC_window[i-15] = OHLC_return[i-15:i-1]    
OHLC_window = OHLC_window[22:]
OHLC_return_train = OHLC_window[:splitpoint]
OHLC_return_test = OHLC_window[splitpoint:]
method_name,method,method_tuning=tuning()

#backtesting
OHLC_selected = OHLC_label[OHLC_label.index.isin(OHLC_hasFeature.index)]
OHLC_backtesting = OHLC_selected[['Open','High','Low','Close','Volume']][splitpoint-2:]
feature_test = feature_idx_sel[splitpoint:]
OHLC_backtesting.index = pd.DatetimeIndex(OHLC_backtesting.index)

### PCA

In [ ]:
pca_df = copy.deepcopy(feature_idx_sel)
pca_df["label"] = OHLC_hasFeature
n_components=4
pca = PCA(n_components = n_components)
components = pca.fit_transform(feature_idx_sel)
labels = {
    str(i): f"PC{i+1}"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(n_components),
    color=pca_df.label,
    title=f'Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)
fig.show()


### Confusion Matrix

In [ ]:
basic, score = confusion(OHLC_hasFeature,feature_idx_sel,splitpoint,display=False)
basic

In [ ]:
score

### train: return

In [ ]:
return_result = training(OHLC_return_train,OHLC_return_test,y_train,y_test,
                method,method_name)
print("return: without tuning")
return_result

In [ ]:
return_result_tuning = training(OHLC_return_train,OHLC_return_test,y_train,y_test,
                method_tuning,method_name)
print("return: with tuning")
return_result_tuning

### train: indicator

In [ ]:
idc_result = training(X_train,X_test,y_train,y_test,
                method,method_name)
print("indicator: without tuning")
idc_result

In [ ]:
idc_result_tuning = training(X_train,X_test,y_train,y_test,
                method_tuning,method_name)
print("indicator: with tuning")
idc_result_tuning

### backtesting

In [ ]:
# SMA Cross
test = Backtest(OHLC_backtesting, SmaCross, cash=100000, commission=0.001,trade_on_close=False)
result = test.run()  
#test.plot(filename=f"./backtest_SMAcross.html")
result

In [ ]:
# single indicator
backtest_single_indicator = batch_test_idc(OHLC_backtesting,feature_test,y_test,name="./backtest_tech.html")
print("backtesting: single indicator")
backtest_single_indicator

In [ ]:
#return: without tuning
backtest_return_result=batch_test_learn(OHLC_backtesting,OHLC_return_train,OHLC_return_test
                                        ,y_train,y_test,method,method_name,name="./backtest_Return.html")
print("backtesting: return (no tuning)")
backtest_return_result

In [ ]:
#return: tuning
backtest_return_result_tuning=batch_test_learn(OHLC_backtesting,OHLC_return_train,OHLC_return_test
                                        ,y_train,y_test,method_tuning,method_name,name="./backtest_Return.html")
print("backtesting: return (with tuning)")
backtest_return_result_tuning

In [ ]:
#indicator: without tuning
backtest_indicator_result=batch_test_learn(OHLC_backtesting,X_train,X_test
                                           ,y_train,y_test,method,method_name,name="./backtest_Indicator.html")
print("backtesting: return (no tuning)")
backtest_indicator_result

In [ ]:
#indicator: tuning
backtest_indicator_result_tuning=batch_test_learn(OHLC_backtesting,X_train,X_test
                                        ,y_train,y_test,method_tuning,method_name,name="./backtest_Indicator.html")
print("backtesting: return (with tuning)")
backtest_indicator_result_tuning